In [7]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES']=''

import sys;
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import json
wiki = json.load(open('/home/kuratov/data/ThePile/Wikipedia/wikipedia-en-0.json', 'r'))

In [9]:
{
    'title': '',
    'text': '',
}

{'title': '', 'text': ''}

In [10]:
# articles to skip:
# List of ...
# {title} may refer to: (disambiguation pages)
# wiki key words to clean
# \nCategory:
# \n\nSee also\n\n
# \n\nReferences\n\n

In [11]:
import re
category_regexp = re.compile(r'\ACategory:[\S ]+\Z')
keywords_regexp = re.compile(r'\n(References|See also|External Links)\s?\n')
title_regexp = re.compile(r'\A[\S ]+?\\n\\n')
list_of_regexp = re.compile(r'\AList of')

def clean_article(text):
    lines = []
    title = ''
    for i, line in enumerate(text.split('\n')):
        if i == 0:
            title = line
            continue
        if category_regexp.match(line):
            continue
        lines += [line]
    text = '\n'.join(lines)
    m = re.search(keywords_regexp, text)
    if m:
        text = text[:m.span()[0]]
    return title, text.strip()

In [12]:
# preprocess wiki
from tqdm import tqdm as tqdm
import json

path = '/home/kuratov/data/ThePile/Wikipedia/wikipedia-en-{}.json'

total_articles = 0
filtered_articles = 0

articles = []

for i in tqdm(range(0, 10)):
    wiki = json.load(open(path.format(i), 'r'))
    for text in wiki:
        total_articles +=1
        title, text = clean_article(text)
        if re.search(list_of_regexp, title):
            filtered_articles += 1
            continue
        if f'{title} may refer to:' in text:
            filtered_articles += 1
            continue
        articles += [{'title': title, 'text': text}]
print(f'{filtered_articles}/{total_articles}')

100%|██████████| 10/10 [04:54<00:00, 29.43s/it]

279906/6033151


In [14]:
print(f'{filtered_articles}/{len(articles)}/{total_articles}')

279906/5753245/6033151


In [15]:
# save shards

from pathlib import Path

shards_path = '/home/kuratov/data/ThePile/Wikipedia/preprocessed_shards'

if not Path(shards_path).exists():
    Path(shards_path).mkdir(parents=True)

import random, math
random.shuffle(articles)
n_shards = 80  # (we possibly have to take n_gpus shards for validation, so we need more shards)

shard_size = math.ceil(len(articles) / n_shards)

for i, start in enumerate(tqdm(range(0, len(articles), shard_size))):
    with (Path(shards_path) / f'shard_{i:02d}.jsonl').open('w', encoding='utf8') as fout:
        print(start, start + shard_size)
        for j in range(start, min(len(articles), start+shard_size)):
            jsoned = json.dumps(articles[j])
            try:
                tmp = json.loads(jsoned)
            except json.JSONDecodeError as e:
                print(f'skipping article {j} in shard {i} cause of JSONDecodeError')
                continue
            fout.write(jsoned+'\n')

  0%|          | 0/80 [00:00<?, ?it/s]

0 71916


  1%|▏         | 1/80 [00:02<03:16,  2.48s/it]

71916 143832


  2%|▎         | 2/80 [00:04<03:13,  2.48s/it]

...

 98%|█████████▊| 78/80 [05:22<00:06,  3.03s/it]

5609448 5681364


 99%|█████████▉| 79/80 [05:24<00:02,  2.89s/it]

5681364 5753280


100%|██████████| 80/80 [05:27<00:00,  4.09s/it]
